In [ ]:
%matplotlib inline
import ROOT, sys, os
from ROOT import std
from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches
from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'
import root_numpy as rn
import pandas as pd
geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

In [ ]:
reco_df = pd.DataFrame(rn.root2rec("../reco_ana.root",treename='LArbysImageTree'))
mc_df   = pd.DataFrame(rn.root2rec("../mc_ana.root",treename='mctree'))

In [ ]:
print len(mc_df), len(reco_df)

In [ ]:
#reco_df.iloc[145]
#print reco_df.iloc[322]['n_vtx3d']
#reco_df

In [ ]:
a=0
data_out = []
mclist = []
pdg2mass={}
pdg2mass[13]=105
pdg2mass[2212]=938
pdg2mass[2112]=939.5
pdg2mass[11]=0#0.5
pdg2mass[311]=497.65
ctor = 0
for entry in xrange(len(mc_df)):
    event = []

    #if(mc_df.iloc[entry]['event']- reco_df.iloc[entry]['event']): continue
    # 1st cut on neutrino energy 100-500 MeV
    if mc_df.iloc[entry]['energyInit']<100 or mc_df.iloc[entry]['energyInit']>500 : continue
    # 2nd cut looking for both visible an invisible V-shape events
    if mc_df.iloc[entry]['daughterPdg_v'].size <2 : continue
    #if mc_df.iloc[entry]['daughterPdg_v'].size ==2 : mclist.append(entry)
    # keep contained events
    for idx, row in mc_df.iterrows():        
        if idx != entry: continue
        init_sum = 0
        dep_sum  = 0 
        proton_length_flag = 0
        for pdg,e_ini in zip(row.daughterPdg_v,row.daughterEnergyInit_v):
            init_sum += e_ini-pdg2mass[pdg]
            #if(pdg == 2212 and e_ini>80): proton_length_flag +=1
            if(pdg == 2212 and e_ini-pdg2mass[pdg]>80): proton_length_flag +=1
            #if idx == 7 and pdg == 2212 : print 'haha', e_ini-pdg2mass[pdg], proton_length_flag
        for pdg,e_dep in zip(row.daughterPdg_v,row.daughterEnergyDep_v):
            dep_sum  += e_dep
        #if (dep_sum/init_sum < 0.9 or proton_length_flag<1): continue
        # 3rd cut on energy deposition
        if dep_sum/init_sum < 0.9 : continue
        # 4th cut on proton length
        if proton_length_flag<1: continue
        ctor+=1        

        #print proton_length_flag, idx, entry

        #print ctor
        #ctor+=1
        
        event.append(entry)
        event.append(row.daughterPdg_v)
        event.append(row.daughterEnergyInit_v)
        event.append(row.daughterEnergyDep_v)
        #print event
        mclist.append(entry)
        data_out.append(event)
print 'data_out size is', len(data_out)
print 'mclist size is ' ,len(mclist)

print mclist
#print data_out
recolist=[]

#for entry in xrange(len(mc_df)):
for entry in mclist :
    #if entry == 322 or entry ==324: continue
    #print entry, reco_df.iloc[entry]['n_vtx3d']
    
    if reco_df.iloc[entry]['n_vtx3d'] >0: 
        flag=0
        #for num_plane in reco_df.iloc[entry]['vtx3d_num_planes_v']:
        for num_plane in reco_df.iloc[entry]['num_planes_v']:
            if num_plane ==3  :flag+=1 
        
        #if flag > 0: recolist.append(entry)
        recolist.append(entry)

print 'recolist size is ',len(recolist)
print recolist
print ctor

recolist size is  28
[8, 9, 23, 29, 42, 52, 55, 65, 93, 107, 114, 119, 125, 142, 145, 150, 
 165, 175, 183, 204, 205, 211, 219, 226, 228, 229, 254, 266]

In [ ]:
r=[]
#print len(mclist)
for entry in mclist:
    for ix,row in mc_df.iterrows():
        if ix != entry: continue
        #print ix
        #print entry,row.daughterPdg_v, row.daughterEnergyInit_v
        for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyInit_v):
            #print pdg
            #print '########'
            #if ix ==186 and pdg ==11 : r.append(e)
            if pdg ==13 : r.append(e)
            
        #print "init:",pdg,e
        #print init_sum
       
    #if dep_sum / init_sum>0.9 : idx.append(ix)
print len(r)
r=np.asarray(r)
#print np.where(r>0.9)[0].size
fig,ax=plt.subplots(figsize=(6,5))

#a,_,_=plt.hist(r,bins=np.arange(0,1.0+0.025,0.025))
a,_,_=plt.hist(r,bins=np.arange(0,500,20))
ax.set_ylim(0,15)
ax.grid()
ax.set_title('%i lepton Ini energy'%len(mclist))

In [ ]:
#Draw distance between 3D vtx
dist=[]
ctr =0
shit=0
for entry in xrange(len(reco_df)):
    #if mc_df.iloc[entry]['daughterPdg_v'].size!=2: continue
    vtx_mc   = [mc_df.iloc[entry]['parentX'],mc_df.iloc[entry]['parentY'],mc_df.iloc[entry]['parentZ']]
    if reco_df.iloc[entry]['vtx3d_x_v'].size<1 : continue
    ctr+=1
    for vtx_idx in xrange(reco_df.iloc[entry]['vtx3d_x_v'].size):
        vtx_reco = [reco_df.iloc[entry]['vtx3d_x_v'][vtx_idx],reco_df.iloc[entry]['vtx3d_y_v'][vtx_idx],
                    reco_df.iloc[entry]['vtx3d_z_v'][vtx_idx]]

        dist_v = ((vtx_mc[0]-vtx_reco[0])**2+(vtx_mc[1]-vtx_reco[1])**2+(vtx_mc[2]-vtx_reco[2])**2)**0.5
        if dist_v<40 : shit+=1
        dist.append(dist_v)
   
    #if(mc_df.iloc[entry]['energyDeposit']>100 and mc_df.iloc[entry]['energyDeposit']<500 ) :shit+=1
    #print entry

fig,ax=plt.subplots(figsize=(5,5))
shit = np.asarray(dist)
#n, bins = np.histogram(shit, 10)
n, bins, patches = plt.hist(shit, 50, normed=1, facecolor='green', alpha=0.5)
ax.set_title("3D vertex Distance[MC-Reco]")
ax.set_xlabel("cm")
#ax.set_xlim(0,10)
print ctr

In [ ]:
#getting V shape events, Nov_8 it is 106
#getting V shape events with a proton, Nov_8 it is 89

recolist=[]
mclist=[]

num_p = 0

for entry in xrange(len(mc_df)):
    if mc_df.iloc[entry]['daughterPdg_v'].size !=2 : continue
    if mc_df.iloc[entry]['daughterPdg_v'].size ==2 : mclist.append(entry)
    if(mc_df.iloc[entry]['event']- reco_df.iloc[entry]['event']): continue
    if reco_df.iloc[entry]['n_vtx3d'] >0: recolist.append(entry)
    
    flag=0
    for x in mc_df.iloc[entry]['daughterPdg_v']:
        if x ==2212: flag+=1
    if flag>0 : num_p+=1
    if flag>1 : print 'shit',entry        
print len(mclist)#, len(recolist)
#print recolist
#print mclist
#print reco_df.index.size
print num_p



In [ ]:
#0) 1 or more reco 3D vertex
#1) atleast 1 vertex in the plane is reconstructed from 3 planes
recolist=[]

#for entry in xrange(len(mc_df)):
for entry in mclist :
    if reco_df.iloc[entry]['n_vtx3d'] >0: 
        flag=0
        for num_plane in reco_df.iloc[entry]['vtx3d_num_planes_v']:
            if num_plane ==3 :flag+=1 
        
        if flag > 0: recolist.append(entry)

print len(recolist)

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))

r=[]
pdg2mass={}
pdg2mass[13]=105
pdg2mass[2212]=938
pdg2mass[2112]=939.5
pdg2mass[11]=0#0.5
pdg2mass[311]=497.65

for ix,row in mc_df.iterrows():
    init_sum=0
    dep_sum=0
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyInit_v):
        init_sum += e-pdg2mass[pdg]
            #print "init:",pdg,e
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyDep_v):
        #if pdg == 2212: dep_sum += e-pdg2mass[pdg]
        dep_sum += e
        #print "dep:",pdg,e
    r.append(-dep_sum + init_sum)
    
r=np.asarray(r)
#print np.where(r>0.9)[0].size
#a,_,_=plt.hist(r,bins=np.arange(0,1.0+0.025,0.025))
a,_,_=plt.hist(r,bins=np.arange(-500,500,10))
#print a.sum()
ax.set_ylim(0,200)
ax.set_title("ini_tot - dep_tot")

In [ ]:
idx=[]
r=[]
pdg2mass={}
pdg2mass[13]=105        #muon
pdg2mass[2212]=938      #proton
pdg2mass[2112]=939.5    #neutron
pdg2mass[11]=0          #electron
pdg2mass[2114]=1232     #delta0
pdg2mass[9010111]=134   #pi0

for ix,row in mc_df.iterrows():
    init_sum=0
    dep_sum=0
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyInit_v):
        init_sum += e-pdg2mass[pdg]
        #print "init:",pdg,e
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyDep_v):
        dep_sum += e
        #print "dep:",pdg,e
    r.append(dep_sum / init_sum)
    if dep_sum / init_sum>0.9 : idx.append(ix)
    
r=np.asarray(r)
print np.where(r>0.9)[0].size
a,_,_=plt.hist(r,bins=np.arange(0,1.0+0.025,0.025))
print a.sum()
print len(idx)

recolist=[]

for entry in idx:
    if reco_df.iloc[entry]['n_vtx3d'] >0: 
        flag=0
        for num_plane in reco_df.iloc[entry]['vtx3d_num_planes_v']:
            if num_plane ==3 :flag+=1 
        
        if flag > 0: recolist.append(entry)

for x in mc_df.iloc[entry]['daughterPdg_v']:
    if x ==2212: num_p+=1
            
print len(recolist)

print recolist


In [ ]:
print pdg2mass

In [ ]:
idx=[]
r=[]
pdg2mass={}
pdg2mass[13]=105        #muon
pdg2mass[2212]=938      #proton
pdg2mass[2112]=939.5    #neutron
pdg2mass[11]=0          #electron
pdg2mass[2114]=1232     #delta0
pdg2mass[9010111]=134   #pi0

for ix,row in mc_df.iterrows():
    if mc_df.iloc[ix]['daughterPdg_v'].size !=2 : continue
    if mc_df.iloc[ix]['daughterPdg_v'].size ==2 : mclist.append(entry)
    if(mc_df.iloc[ix]['event']- reco_df.iloc[ix]['event']): continue
    if reco_df.iloc[ix]['n_vtx3d'] >0: recolist.append(entry)
    
    init_sum=0
    dep_sum=0
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyInit_v):
        
        if pdg ==2212 : init_sum = (e-pdg2mass[pdg])/7.5
        #print "init:",pdg,e
        #print init_sum
    r.append(init_sum)
    #if dep_sum / init_sum>0.9 : idx.append(ix)
    
r=np.asarray(r)
#print np.where(r>0.9)[0].size

#a,_,_=plt.hist(r,bins=np.arange(0,1.0+0.025,0.025))
a,_,_=plt.hist(r,bins=np.arange(0,500,1))
ax.set_ylim(0,10)


In [ ]:
#Here we have selected events with >90% energy depostion and also the shorter track
r=[]
pdg2mass={}
pdg2mass[13]=105        #muon
pdg2mass[2212]=938      #proton
pdg2mass[2112]=939.5    #neutron
pdg2mass[11]=0          #electron
pdg2mass[2114]=1232     #delta0
pdg2mass[9010111]=134   #pi0

for ix,row in mc_df.iterrows():
    init_sum=0
    dep_sum=0
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyInit_v):
        init_sum += e-pdg2mass[pdg]
        #print "init:",pdg,e
        
    for pdg,e in zip(row.daughterPdg_v,
                     row.daughterEnergyDep_v):
        dep_sum += e
        #print "dep:",pdg,e
    r.append(dep_sum / init_sum)
    
r=np.asarray(r)
#print np.where(r>0.9)[0].size
a,_,_=plt.hist(r,bins=np.arange(0,1.0+0.025,0.025))
#print a.sum()


recolist_0 = [8, 9, 23, 29, 42, 52, 55, 65, 93, 107, 114, 119, 125, 142, 145, 150, 165, 175, 
              183, 204, 205, 211, 219, 226, 228, 229, 254, 266]

In [ ]:
#reco_df

In [ ]:
a = np.asarray(reco_df.iloc[8].num_pixel_frac_vv[0])
print a
a.sum()

In [ ]:
recolist_0 = [8, 9, 23, 29, 42, 52, 55, 65, 93, 107, 114, 119, 125, 142, 145, 150, 165, 
              175, 183, 204, 205, 211, 219, 226, 228, 229, 254, 266]
for plane in xrange(3):
    vv=[]
    for entry in recolist_0:
        v_higer = []
        v_sum = 0
        for v in reco_df.iloc[entry].num_pixel_frac_vv:
            if reco_df.iloc[entry].num_pixel_frac_vv.size == 1:
                v_higer = v
            if reco_df.iloc[entry].num_pixel_frac_vv.size > 1:
                v_np = np.asarray(v)
                if v_np.sum() > v_sum : 
                    v_higer = v
                    v_sum = v_np.sum()
            vv.append(v)

for v in vv:
    print v

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0)                                                                          
iom.add_in_file("../out_0000_0099.root")
iom.add_in_file("../ccqe_0100_0199.root")
iom.add_in_file("../ccqe_0200_0299.root")
iom.set_out_file("/tmp/trash.root")
iom.initialize()

larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("empty.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

In [ ]:
#mclist    = [8, 9, 23, 29, 42, 52, 55, 64, 65, 85, 93, 104, 107, 114, 119, 125, 139, 142, 145, 150, 165, 167, 175, 183, 204, 205, 211, 219, 226, 228, 229, 254, 255, 264, 266]
#recolist  = [8, 9, 23, 52, 65, 93, 107, 114, 125, 142, 145, 150, 175, 204, 205, 211, 219, 226, 228, 229, 266]

In [ ]:
print recolist
print len(recolist)
print len(mclist)

In [ ]:
reco_df.iloc[10]
#mc_df

In [ ]:
#print data_out[0]
recolist_0 = [8, 9, 23, 29, 42, 52, 55,
              65, 93, 107, 114, 119, 125, 142, 145, 150, 165, 175, 183, 204, 205, 211, 219, 226, 228, 229, 254, 266]

In [ ]:
print len(mclist)
print mclist

In [ ]:
print len(recolist)

In [ ]:
colormap=['blue','red','magenta','green','orange','yellow','pink']
faillist = []
a=[10]
#for entry in reco_idx.values:
for entry in mclist:
#for entry in a :
    iom.read_entry(entry)
    larbysimg.process(iom)
    mgr=larbysimg.Manager()
    
    img_v = []
###################################
    fig,ax=plt.subplots(figsize=(20,20))
    maxx=0;minx=9999
    maxy=0;miny=9999
    plane=0
    caca_img=np.zeros((485, 485,3))
    cc=-1
    for mat in mgr.InputImages():
        cc+=1
        
        img_v.append(pygeo.image(mat))
        shape_img=np.where(img_v[plane]>1.5,1.0,0.0).astype(np.uint8)

        plane+=1

        nz_pixels=np.where(shape_img>0.0)
        
        maxx=np.maximum(maxx,np.max(nz_pixels[1])+100)
        minx=np.minimum(minx,np.min(nz_pixels[1])-100)

        maxy=np.maximum(maxy,np.max(nz_pixels[0])+100)
        miny=np.minimum(miny,np.min(nz_pixels[0])-100)

        caca_img[:,:,cc]=shape_img
    caca_img=np.where(caca_img>0.0,1.0,255)
    caca_img=caca_img.astype(np.uint8)
    plt.imshow(caca_img,interpolation='none')
#####################################
    
    for iz in xrange(3):
        w_v=mc_df.iloc[entry].vtx2d_w
        t_v=mc_df.iloc[entry].vtx2d_t
        
        for w,t in zip(w_v,t_v):
            plt.plot(t,w,'*',color='yellow',markersize=20)
    
    
    #for iy in xrange(reco_df.iloc[entry].n_circle_vtx):
    #print 'shit ',reco_df.iloc[entry].circle_vtx_x_vv.size
    for iy in xrange(reco_df.iloc[entry].circle_vtx_x_vv.size):
        x_v=reco_df.iloc[entry].circle_vtx_x_vv[iy]
        y_v=reco_df.iloc[entry].circle_vtx_y_vv[iy]

        for x,y in zip(x_v,y_v):
            plt.plot(x,y,'*',color='orange',markersize=20)
        
        #w_v=mc_df.iloc[entry].vtx2d_w
        #t_v=mc_df.iloc[entry].vtx2d_t
        
        #for w,t in zip(w_v,t_v):
            #plt.plot(t,w,'*',color='yellow',markersize=20)
    
    v_higer = [0,0,0]
    v_sum = 0
    for v in reco_df.iloc[entry].num_pixel_frac_vv:
        if reco_df.iloc[entry].num_pixel_frac_vv.size == 1:
            v_higer = v
        if reco_df.iloc[entry].num_pixel_frac_vv.size > 1:
            v_np = np.asarray(v)
            if v_np.sum() > v_sum : 
                v_higer = v
                v_sum = v_np.sum()
    v_higer = [0,0,0]
    v_sum = 0
    num_vtx_p = 0 
#########################################
    for v, num_p in zip(reco_df.iloc[entry].num_pixel_frac_vv, reco_df.iloc[entry].num_planes_v):
        if reco_df.iloc[entry].num_pixel_frac_vv.size == 1:
            v_higer = v
            num_vtx_p = num_p
        if reco_df.iloc[entry].num_pixel_frac_vv.size > 1:
            v_np = np.asarray(v)
            if v_np.sum() > v_sum : 
                v_higer = v
                v_sum = v_np.sum()
                num_vtx_p = num_p
#########################################
    reco_flag ='pass'
    if reco_df.iloc[entry].circle_vtx_x_vv.size <1 : 
        reco_flag = 'fail'
        faillist.append(entry)
    #print v_higer
    
    #print mc_df.iloc[entry].daughterTrackid_v
    #print mc_df.iloc[entry].daughterParentTrackid_v
    if mc_df.iloc[entry].daughterPdg_v.size ==2 :
        #plt.text(-100,100,"num_p_frac:\n %f,\n %f,\n %f \n daughter pdg:\n %i, %dMeV, tID: %i, tpID: %i  \n %i, %dMeV , tID: %i, tpID: %i"%
        plt.text(-100,100,"%s,\n nu initial eng: %fMev, \n num_vtx_p: %i \n num_p_frac:\n %f,\n %f,\n %f \n daughter pdg:\n %i, %dMeV, tID: %i, tpID: %i  \n %i, %dMeV , tID: %i, tpID: %i"%

                 (reco_flag,
                  mc_df.iloc[entry].energyInit,
                  num_vtx_p,
                  v_higer[0],v_higer[1],v_higer[2],
                  mc_df.iloc[entry].daughterPdg_v[0],
                  mc_df.iloc[entry].daughterEnergyDep_v[0],
                  mc_df.iloc[entry].daughterTrackid_v[0],
                  mc_df.iloc[entry].daughterParentTrackid_v[0],
                  mc_df.iloc[entry].daughterPdg_v[1],
                  mc_df.iloc[entry].daughterEnergyDep_v[1],
                  mc_df.iloc[entry].daughterTrackid_v[1],
                  mc_df.iloc[entry].daughterParentTrackid_v[1]
                 ),fontsize = 30)
    if mc_df.iloc[entry].daughterPdg_v.size ==3 :
        plt.text(-100,100,"%s,nu initial eng: %fMev,\n num_vtx_p: %i \n num_p_frac: \n%f,\n %f,\n %f \n daughter pdg:\n %i, %dMeV , tID: %i, tpID: %i\n %i, %dMeV , tID: %i, tpID: %i\n %i, %dMeV , tID: %i, tpID: %i "%(
                reco_flag,
                mc_df.iloc[entry].energyInit,
                num_vtx_p,
                v_higer[0],v_higer[1],v_higer[2],
                mc_df.iloc[entry].daughterPdg_v[0],
                mc_df.iloc[entry].daughterEnergyDep_v[0],
                mc_df.iloc[entry].daughterTrackid_v[0],
                mc_df.iloc[entry].daughterParentTrackid_v[0],
                mc_df.iloc[entry].daughterPdg_v[1],
                mc_df.iloc[entry].daughterEnergyDep_v[1], 
                mc_df.iloc[entry].daughterTrackid_v[1],
                mc_df.iloc[entry].daughterParentTrackid_v[1],
                mc_df.iloc[entry].daughterPdg_v[2],
                mc_df.iloc[entry].daughterEnergyDep_v[2],
                mc_df.iloc[entry].daughterTrackid_v[2],
                mc_df.iloc[entry].daughterParentTrackid_v[2]
            ),fontsize = 30)

    ax.set_title("Entry {}".format(entry),fontweight='bold',fontsize=40)
    plt.savefig("reco_%04d.png"%entry)
    #break
    plt.show()

In [ ]:
reco_df_copy = reco_df.copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_df_copy = reco_df_copy.query(cut)

In [ ]:
reco_df_copy = reco_df_copy[reco_df_copy['vtx3d_num_planes_v'].apply(lambda x : np.where(x==3)[0].size)>0]

In [ ]:
def pixels(x):
    y=np.hstack(x)
    return np.where(y>0.9)[0].size

reco_idx=reco_df_copy[reco_df_copy['num_pixel_frac_vv'].apply(pixels) > 0].index

In [ ]:
np3_v=reco_df.ix[reco_idx].vtx3d_num_planes_v.apply(lambda x : np.where(x==3)[0][0]).values

In [ ]:
dw_v=[]
dt_v=[]
for ix,idx in enumerate(reco_idx):
    mc_t   = mc_df.ix[idx].vtx2d_w
    mc_w   = mc_df.ix[idx].vtx2d_t
    reco_w = reco_df.ix[idx].circle_vtx_x_vv[np3_v[ix]]
    reco_t = reco_df.ix[idx].circle_vtx_y_vv[np3_v[ix]]
    dw = mc_w - reco_w
    dt = mc_t - reco_t
    dw_v.append(dw)
    dt_v.append(dt)
dw_v = np.array(dw_v)
dt_v = np.array(dt_v)

In [ ]:
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dw_v[:,i],label='Plane {}'.format(i),bins=np.arange(-80,80,2),alpha=0.5)
ax.set_xlabel("mc_w - reco_w [pixel]",fontweight='bold')
ax.legend()
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dt_v[:,i],label='Plane {}'.format(i),bins=np.arange(-80,80,2),alpha=0.5)
ax.set_xlabel("mc_t - reco_t [pixel]",fontweight='bold')
ax.legend()
plt.grid()
plt.show()


for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(np.sqrt(np.power(dw_v[:,i],2)+np.power(dt_v[:,i],2)),
            label='Plane {}'.format(i),bins=np.arange(0,100,1),alpha=0.5,lw=2)
    ax.set_xlabel("dist 2D mc->reco [pixel]",fontweight='bold')
    ax.set_ylim(0,15)
    ax.legend()
    plt.grid()
    plt.show()
